<a href="https://colab.research.google.com/github/lhslab/tech-challenge-fase4-lh/blob/main/MelhorNotebook-lh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface insightface tqdm onnxruntime mediapipe scikit-learn numpy

In [19]:
import cv2
from deepface import DeepFace
import os
from tqdm import tqdm
import mediapipe as mp
from insightface.app import FaceAnalysis
import uuid
from sklearn.ensemble import IsolationForest
import numpy as np
import pandas as pd
from mediapipe.python.solutions.hands import Hands


In [20]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

In [21]:
def generate_summary(emotion_counts, total_frames, output_path):
    # Exibir Relatório no Console
    print("RELATÓRIO:")

    # Exibir o total de frames analisados
    print(f"\nTotal de frames analisados: {total_frames}")

    # Exibir o resumo das emoções detectadas
    print("\nResumo das emoções detectadas:")
    for emotion, count in emotion_counts.items():
        print(f"{emotion}: {count} ocorrências")

    # Exibir o número de anomalias detectadas (categorias 'Unknown')
    anomalies = emotion_counts.get("Unknown", 0)
    print(f"\nNúmero de anomalias detectadas: {anomalies}")

    # Criar o arquivo de relatório
    report_path = os.path.join(os.path.dirname(output_path), 'relatorio_analise.txt')

    with open(report_path, 'w') as report_file:
        # Escrever o total de frames analisados
        report_file.write(f"Total de frames analisados: {total_frames}\n")

        # Escrever o resumo das emoções detectadas
        report_file.write("\nResumo das emoções detectadas:\n")
        for emotion, count in emotion_counts.items():
            report_file.write(f"{emotion}: {count} ocorrências\n")

        # Escrever o número de anomalias detectadas
        report_file.write(f"\nNúmero de anomalias detectadas: {anomalies}\n")

    print(f"\nRelatório salvo em: {report_path}")

In [22]:
def criar_pasta_para_rostos(diretorio="detected_faces"):
    # Obter o caminho absoluto
    caminho_absoluto = os.path.abspath(diretorio)

    # Criar a pasta se ela não existir
    if not os.path.exists(caminho_absoluto):
        os.makedirs(caminho_absoluto)
        print(f"Pasta criada: {caminho_absoluto}")
    else:
        print(f"Pasta já existente: {caminho_absoluto}")
        return caminho_absoluto

# Exemplo de uso
pasta_rostos = criar_pasta_para_rostos()

def ajustar_limites_rosto(box, width, height):

    x1, y1, x2, y2 = box
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(width, x2), min(height, y2)
    return x1, y1, x2, y2
def rastrear_ou_criar_id(box, face_trackers, proximo_id):

    for existing_id, tracker_info in face_trackers.items():
        tracked_box, _ = tracker_info
        iou = sobreposicao(box, tracked_box)
        if iou > 0.5:
            return existing_id, proximo_id, face_trackers

    face_id = proximo_id
    proximo_id += 1
    return face_id, proximo_id, face_trackers
#face_roi, face_id, Rastreados, diretorio_saida

def salvar_rosto(face_roi, face_id, Rastreados, diretorio_saida):

    if face_id not in Rastreados:
        Rastreados.add(face_id)
        face_image_path = os.path.join(diretorio_saida, f"face_{face_id}.jpg")
        cv2.imwrite(face_image_path, cv2.cvtColor(face_roi, cv2.COLOR_RGB2BGR))

def sobreposicao(boxA, boxB):
    # Calcular a interseção sobre união (IOU) entre dois retângulos
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou


def salvar_excel(emotions_per_id, output_path):
    # Converter o dicionário em um DataFrame
    data = [{"ID": face_id, "Emoção Predominante": emotion} for face_id, emotion in emotions_per_id.items()]
    df = pd.DataFrame(data)

    # Caminho para o arquivo Excel
    excel_path = os.path.join(os.path.dirname(output_path), 'analise_emocoes.xlsx')

    # Salvar o DataFrame no Excel
    df.to_excel(excel_path, index=False)
    print(f"Arquivo Excel salvo em: {excel_path}")

Pasta criada: /content/detected_faces


In [ ]:
def is_hand_writing_or_holding(hand_landmarks):
    """
    Detecta se uma mão está escrevendo ou manipulando algo baseado na proximidade dos dedos.
    """
    # Coordenadas dos landmarks relevantes
    thumb_tip = hand_landmarks[4]   # Ponta do polegar
    index_tip = hand_landmarks[8]  # Ponta do dedo indicador
    middle_tip = hand_landmarks[12]  # Ponta do dedo médio
    ring_tip = hand_landmarks[16]  # Ponta do dedo anelar
    pinky_tip = hand_landmarks[20]  # Ponta do dedo mínimo

    # Distâncias entre polegar e os outros dedos
    dist_thumb_index = ((thumb_tip.x - index_tip.x)**2 + (thumb_tip.y - index_tip.y)**2)**0.5
    dist_thumb_middle = ((thumb_tip.x - middle_tip.x)**2 + (thumb_tip.y - middle_tip.y)**2)**0.5
    dist_thumb_ring = ((thumb_tip.x - ring_tip.x)**2 + (thumb_tip.y - ring_tip.y)**2)**0.5
    dist_thumb_pinky = ((thumb_tip.x - pinky_tip.x)**2 + (thumb_tip.y - pinky_tip.y)**2)**0.5

    # Critério: todos os dedos próximos ao polegar indicam manipulação
    threshold = 0.05  # Ajustar conforme o tamanho normalizado
    is_holding = (
        dist_thumb_index < threshold and
        dist_thumb_middle < threshold and
        dist_thumb_ring < threshold and
        dist_thumb_pinky < threshold
    )

    return is_holding


In [25]:
def detect_emotions(video_path, output_path):
    # Capturar vídeo do arquivo especificado
    cap = cv2.VideoCapture(video_path)

    # Verificar se o vídeo foi aberto corretamente
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Obter propriedades do vídeo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Definir o codec e criar o objeto VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Inicializar dicionário para contabilizar emoções
    emotion_counts = {}
    emotions_per_id = {}
    pose_data = []
    anomalies_detected = 0
    total_frames_analyzed = 0
    frames_sem_rosto = 0
    frames_so_maos = 0
    frames_maos_manipulando = 0

    #prev_gray = None  # Frame anterior em escala de cinza

    app = FaceAnalysis()
    app.prepare(ctx_id=0)  # Use ctx_id=-1 for CPU

    diretorio_saida="detected_faces"

    criar_pasta_para_rostos(diretorio=diretorio_saida)

    Rastreador = {}
    proximo_id = 1
    Rastreados = set()

    isolation_forest = IsolationForest(contamination=0.1)

    maos = Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
    maos_sobre_a_face = 0
    # Loop para processar cada frame do vídeo
    for _ in tqdm(range(total_frames), desc="Processando vídeo"):
        # Ler um frame do vídeo
        ret, frame = cap.read()

        # Se não conseguiu ler o frame (final do vídeo), sair do loop
        if not ret:
            break

        rgb_frame = frame[:, :, ::-1]
        faces = app.get(rgb_frame)
        hands_results = maos.process(rgb_frame)

        results = pose.process(rgb_frame)


        #trecho luizhg
        # Para landmarks de pose:
        if results.pose_landmarks:
          mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)







        if len(faces) == 0:
            frames_sem_rosto += 1

            # Verificar se há mãos detectadas
            if hands_results.multi_hand_landmarks:
                frames_so_maos += 1
                cv2.putText(frame, "Hands only detected", (10, height - 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

                if is_hand_writing_or_holding(hands_results):
                  frames_maos_manipulando += 1
                  cv2.putText(frame, "Hands holding detected", (10, height - 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

            elif results.pose_landmarks:
                # Verificar landmarks dos braços (cotovelos e pulsos)
                landmarks = results.pose_landmarks.landmark
                left_elbow = landmarks[13]  # Cotovelo esquerdo
                right_elbow = landmarks[14]  # Cotovelo direito
                left_wrist = landmarks[15]  # Pulso esquerdo
                right_wrist = landmarks[16]  # Pulso direito

                # Calcular posições absolutas no frame
                arm_landmarks = [
                    (left_elbow.x, left_elbow.y),
                    (right_elbow.x, right_elbow.y),
                    (left_wrist.x, left_wrist.y),
                    (right_wrist.x, right_wrist.y),
                ]

                arm_positions = [
                    (int(lm[0] * width), int(lm[1] * height)) for lm in arm_landmarks
                ]

                # Marcar braços nos frames
                for pos in arm_positions:
                    cv2.circle(frame, pos, 5, (0, 255, 0), -1)

                cv2.putText(frame, "Arms only detected", (10, height - 80),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Exibir contagem de frames no vídeo
        #cv2.putText(frame, f"No Faces: {frames_sem_rosto}, Hands Only: {frames_so_maos}",(10, height - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(frame, f"No Faces: {frames_sem_rosto}, Hands Only: {frames_so_maos}, Writing: {frames_maos_manipulando}", (10, height - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)




        # Manter o controle do número de faces detectadas
        if 'prev_num_faces' not in locals():
            prev_num_faces = 0

        num_faces = len(faces)

        faces_text = f"Number of faces changed from {prev_num_faces} to {num_faces}"
        cv2.putText(frame, faces_text, (10, height - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        if num_faces != prev_num_faces:
            prev_num_faces = num_faces

        for face in faces:
            box = face.bbox.astype(int)
            x1, y1, x2, y2=ajustar_limites_rosto(box, width, height)

            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)

            face_roi = rgb_frame[y1:y2, x1:x2]

            ja_existe = False
            id_face = None




            if face_roi.size == 0:
                print(f"Rosto inválido detectado no frame {_}, ignorando.")
                continue
            else:
                face_id, proximo_id, Rastreador = rastrear_ou_criar_id(box, Rastreador, proximo_id)
                Rastreador[face_id] = ((x1, y1, x2, y2), _)
                salvar_rosto(face_roi, face_id, Rastreados, diretorio_saida)

            try:
                # Chamar a função para detectar anomalias
                #anomalies_detected = detectar_anomalia_mao(
                #    prev_gray, face_roi, x1, y1, frame, anomalies_detected
                #)

                analysis = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)

                if isinstance(analysis, list) and len(analysis) > 0:
                    dominant_emotion = analysis[0]['dominant_emotion']
                else:
                    dominant_emotion = "Unknown"

            except Exception as e:
                dominant_emotion = "Error"
                print(f"Error analyzing emotion: {e}")

            # Atualizar contador de emoções
            if dominant_emotion not in emotion_counts:
                emotion_counts[dominant_emotion] = 0
            emotion_counts[dominant_emotion] += 1

            emotions_per_id[face_id] = dominant_emotion

            cv2.putText(frame, f"ID:{face_id} - {dominant_emotion}",(box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX,0.8, (0, 255, 255), 2)

            if hands_results.multi_hand_landmarks:
                      for hand_landmarks in hands_results.multi_hand_landmarks:
                          hand_points = [(int(landmark.x * width), int(landmark.y * height))
                                        for landmark in hand_landmarks.landmark]

                          for point in hand_points:
                              px, py = point
                              if x1 <= px <= x2 and y1 <= py <= y2:
                                  maos_sobre_a_face += 1
                                  cv2.putText(frame, "Hand over face detected", (10, height - 50),
                                              cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 165, 255), 2)
                                  break

    # Exibir contagem no frame
            cv2.putText(frame, f"Hands over face: {maos_sobre_a_face}", (10, height - 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)



            #prev_gray = gray_frame  # Atualizar o frame anterior

        # Escrever o frame processado no arquivo de vídeo de saída
        out.write(frame)

        # Analisar dados de pose para detectar anomalias
    if len(pose_data) > 10:
        pose_data = np.array(pose_data)
        anomaly_labels = isolation_forest.fit_predict(pose_data)
        anomalies_detected = np.sum(anomaly_labels == -1)

    with open(os.path.join(os.path.dirname(output_path), 'relatorio_analise.txt'), 'a') as report_file:
        report_file.write(f"\nNúmero de anomalias detectadas (movimentos anômalos): {anomalies_detected}\n")

    # Chamar a função para gerar o relatório
    generate_summary(emotion_counts, total_frames, output_path)

    salvar_excel(emotions_per_id, output_path)

    # Liberar a captura de vídeo e fechar todas as janelas
    cap.release()
    out.release()
    #cv2.destroyAllWindows()

In [26]:
script_dir = os.getcwd()
input_video_path = "/content/video_cut-original-3seg.mp4"
output_video_path = "/content/output_video_16.mp4"  # Nome do vídeo de saída

# Chamar a função para detectar emoções no vídeo e salvar o vídeo processado
detect_emotions(input_video_path, output_video_path)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
Pasta criada

Processando vídeo: 100%|██████████| 90/90 [04:11<00:00,  2.79s/it]

RELATÓRIO:

Total de frames analisados: 90

Resumo das emoções detectadas:
sad: 117 ocorrências
neutral: 95 ocorrências
angry: 75 ocorrências
fear: 60 ocorrências
happy: 12 ocorrências
surprise: 1 ocorrências

Número de anomalias detectadas: 0

Relatório salvo em: /content/relatorio_analise.txt
Arquivo Excel salvo em: /content/analise_emocoes.xlsx
